# <center> Convolutional Neural Networks</center>

<center>This notebook is a part of teaching material for CS-EJ3311 - Deep Learning with Python</center>
<center>Aalto University (Espoo, Finland)</center>
<center>fitech.io (Finland)</center>

# <center>Step1. Data</center>

[Fashion-MNIST ](https://www.tensorflow.org/datasets/catalog/fashion_mnist) dataset consists of data points representing articles of an online shop. Each article is characterized by a $28 \times 28$ pixels grayscale image. Moreover, each article is associated with a label $y$ that indicates to which of $10$ classes (or product categories) this article belongs. 

In [ ]:
#@title  Import Python libraries

import numpy as np                  # library for numerical computations (vectors, matrices, tensors)
import pandas as pd                 # library for handling data
import matplotlib.pyplot as plt     # library providing tools for plotting data 
import tensorflow as tf             # end-to-end open source platform for deep learning
from tensorflow.keras import layers # layers are the basic building blocks of neural networks in Keras

In [ ]:
#@title Load Data

# load dataset
from tensorflow.keras.datasets import fashion_mnist
(trainval_images, trainval_labels), (test_images, test_labels) = fashion_mnist.load_data()

# shape of train and test image
print(f'Number of training and validation examples {trainval_images.shape}')
print(f'Number of test examples {test_images.shape}')

# the label values are stored as integer numbers, in the range [0, 9]
# these numeric labels correspond to the classes of clothing items the image represent:

labels = np.unique(test_labels)



In [ ]:
#@title Preprocess data

# Choose only a subset for training data in order to reduce training time:

# select subset of trainval_images and trainval_labels
X_trainval = trainval_images[:16000]
y_trainval = trainval_labels[:16000]

# select whole test set
X_test = test_images
y_test = test_labels

#  Reshape feature matrices

X_trainval = X_trainval.reshape(-1, 28, 28, 1)
X_test = test_images.reshape(-1, 28, 28, 1)

# When training the ANN it is a good practice to normalize the input values so that they are between 0 and 1, in our case, the pixel values.
# Let's transform feature values of type uint8 in a range [0, 255] to feature values of type float in the range [0, 1]:

# Normalize data to have feature values between 0 and 1
X_trainval = X_trainval/ 255.0
X_test = X_test/ 255.0 

# <center>Step 2. Define CNN Structure</center>

In [ ]:
#@title Define model architecture

# define the model architecture
model = tf.keras.models.Sequential([
    # input + (Conv → Conv → Pool) block
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu", input_shape=[28,28,1], name='cv1'),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu", name='cv2'),
    tf.keras.layers.MaxPool2D(pool_size=2, name='maxpool'),
    # Flatten 
    tf.keras.layers.Flatten(name='flatten'),
    # 1st Dense layer
    tf.keras.layers.Dense(64,"relu", name='dense'),
    # 2nd Dense (output) layer
    tf.keras.layers.Dense(10, activation="softmax", name='output')
])

tf.keras.utils.plot_model(
    model,
    show_shapes=True, 
    show_layer_names=True
    )

# <center>Step 3. Choose Loss Function and Optimizer</center>

In [ ]:
#@title Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='RMSprop',
              metrics="sparse_categorical_accuracy")


# <center>Step 4. Training - Adjusting CNN Weights </center>



In [ ]:
#@title Train the model

history = model.fit(X_trainval, y_trainval, validation_split=0.2, batch_size=32, epochs=10, verbose=1)

In [ ]:
#@title Plot training log

pd.DataFrame(history.history).plot(figsize=(10,5))
plt.grid(True)
plt.xlabel('epoch', fontsize=14)
plt.show()

In [ ]:
#@title Evaluation on Test Set

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test dataset:', test_accuracy)

# <center>Step 5. Visualizing the activation maps of  convolutional layers </center>



In [ ]:
#@title Plot activation maps of the 1s.t convolutional layer

# input layer
in_layer = model.input
# all other layers
layers = [layer.output for layer in model.layers]

# create a model 
activation_model = tf.keras.models.Model(inputs = in_layer, outputs = layers)

# this is the image whose feature map we will visualize
plt.imshow(X_test[0].reshape(28,28), cmap='gray')
plt.show()

# pass input and get feature maps of the image
activation = activation_model(X_test[0].reshape(1, 28, 28, 1))

first_layer_activation = activation[0] 

# visuale activation maps of the first convolutional layer
plt.figure(figsize=(16,16))

for i in range(first_layer_activation.shape[-1]):
    plt.subplot(8,8,i+1)
    plt.axis('off') # remove ticks
    plt.imshow(first_layer_activation[0, :, :, i], cmap='gray')
    plt.title('act. map '+ str(i+1))

plt.show()

In [ ]:
#@title Plot activation maps of the 3rd convolutional layer

third_layer_activation = activation[2] 

# visuale activation maps of after max pooling layer
plt.figure(figsize=(16,16))

for i in range(third_layer_activation.shape[-1]):
    plt.subplot(8,8,i+1)
    plt.axis('off') # remove ticks
    plt.imshow(third_layer_activation[0, :, :, i], cmap='gray')
    plt.title('act. map '+ str(i+1))

plt.show()